# Using *owimetadatabase-preprocessor* to retrieve the data from Owimetadatabase

If you haven't already, you need to install the following packages in your Python environment or if you want to update them (mostly owimetadatabse-preprcoessor as it is frequently updated) (remove *%%capture* here if you have problems when installing):

In [ ]:
%%capture
%pip install python-dotenv==1.0.0
%pip install owimetadatabase_preprocessor

Load necessary modules:

In [ ]:
import os

from owimetadatabase_preprocessor.locations.io import LocationsAPI 
from owimetadatabase_preprocessor.geometry.io import GeometryAPI
from owimetadatabase_preprocessor.soil.io import SoilAPI
from owimetadatabase_preprocessor.fatigue.io import FatigueAPI

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
from dotenv import load_dotenv

load_dotenv()

For authorization, the recommended way is to store your access token securely for authentication locally as an environment variable (can be in *.env* file next to your code, e.g. *.env* file with *OWIMETADB_TOKEN=<your-token-here>* field). Otherwise, just copying it to the **TOKEN** variable also works (but be careful with sahring/publishing, delete it from the notebook before). 

To load it into variable securely with your *.env* file:

In [ ]:
TOKEN = os.getenv('OWIMETADB_TOKEN')

Otherwise, just copy paste the provided token into **TOKEN**. Do not forget to delete it from your code if sharing!

In [ ]:
TOKEN = "<your-token-string-goes-here>"

**TOKEN** can be passed down to your API to authenticate when requesting data from *owimetadatabase*. You can do this directly by passing *token* argument or passing using *header* in the format *{"Authorization": f"Token {TOKEN}"}*. You can also specify endpoint URL yourself if needed but the most up-to-date one is already provided by default.

**Please note that you will need to change the input arguments in this notebook according to what you have access to, so it will actually provide an output.**

### Locations API

For example, we can start working with locations, and for this we would need to work with *LocationsAPI*.

In [ ]:
api_loc = LocationsAPI(token=TOKEN)

To view all the projects you have access to:

In [ ]:
data_projectsites = api_loc.get_projectsites()
data_projectsites["data"]

Here and further, the data is provided in dictionary format, with "exists" key specifying if queried data exists and "data" key with data itslef in a suitable format (dataframes).

In [ ]:
list(data_projectsites.keys())

To get all the location information for the specified projectsite: 

In [ ]:
locs = api_loc.get_assetlocations(projectsite="Nobelwind")

To make sure this data exists:

In [ ]:
locs["exists"]

To view five first rows of the locations dataframe:

In [ ]:
locs["data"].head(3)

For example, if there is no data for the specified project:

In [ ]:
locs_false = api_loc.get_assetlocations(projectsite="Somename")
locs_false["exists"]

Please note that if you have access to a lot of projects/assets, it is better to narrow down your query as much as possible, e.g. by specifying a projectsite name or even turbine name(s). Otherwise, it might result in a lot of data and the database might run into timeout with no output returned. You even might need to use more specific method from the ones offered by the package. For that see more in documentation.

In [ ]:
data_asset = api_loc.get_assetlocation_detail(projectsite="Nobelwind", assetlocation=["BBG01"])
data_asset["data"]

Alternatively, you can request several turbines at once. Can be from different projects. Since the requests right now are done sequentially, timeout is not as crucial in this case as it might be for geometry queries.

In [ ]:
data_asset = api_loc.get_assetlocations(assetlocations=["BBG01", "NRTA1"])
data_asset["data"]

You can also plot the locations for the all turbines you have access to, e.g. for a specific project or a set of specific turbines in a list:

In [ ]:
api_loc.plot_assetlocations(projectsite="Nobelwind")

Please refer to the documentation for more specific details of each method and more capabilities. The package still might expand and add more capabilities in terms of querying specific data!

### Geometry API

This more extensive part of the package allows to gather and process geometrical data for each existing turbine in the database. It works in a similar manner to locations to get "raw" database information. But it gets a little bit different in terms of having methods allowing some preprocessing to get important geometry information (height, etc.) which can be used, e.g., as input to FE models. 

In [ ]:
api_geo = GeometryAPI(token=TOKEN)

To load turbine processor to calculate the information for the turbine(s) (note it might take some time for multiple turbines and it can even timeout sometimes, please rerun the cell in this case):

In [ ]:
turbines = ["BBG01", "BBG10"]
owts = api_geo.get_owt_geometry_processor(turbines)

By running the next cell you can see the information it can provide/store/calculate (mostly in dataframes or dictionaries): 

In [ ]:
list(owts.__dict__.keys())

You can already acces the most trivial information like water depth in dictionary format:

In [ ]:
owts.water_depth

But the most important method is to calculate the information(s) about turbine(s) into dataframes. If you try to query some dataframes without running processing, they will provide no information, e.g.:

In [ ]:
owts.all_turbines

Hence, you would want to run the processing explicitly:

In [ ]:
owts.process_structures()

After this you can query all kinds of dataframes, e.g. all turbines general information:

In [ ]:
owts.all_turbines

Only tower geometry (tubular structures) for all turbines:

In [ ]:
owts.tower

For a specific turbine (you can either specify the name directly or the number in the list of turbines provided before):

In [ ]:
owts.select_owt("BBG01").tower

Or even all tubular sections for all subassemblies for all turbines (convenient to filter later according to your requirement):

In [ ]:
owts.all_tubular_structures

Of course, you can also query other information like RNA:

In [ ]:
owts.rna

Or lumped masses, etc.

In [ ]:
owts.all_lumped_mass

### Soil API

Geotechnical part mainly consists of many *get_\*, \*_exists* and some *plot_\** methods used to retrieve data in a suitable and convenient format from the database. To start, as usual, we first initiate the specific API object with your credentials.

In [ ]:
api_geo = SoilAPI(token=TOKEN)

Now, we can start working with it. Since it is essentially a lot of similar methods but for different types of data, here, we will show only a few of them as examples, the rest can be consulted in the [documentation](https://owi-lab.github.io/owimetadatabase-preprocessor/index.html). 

To start, let's see what survey campaigns there are:

In [ ]:
survey_campaigns = api_geo.get_surveycampaigns()
survey_campaigns["data"]

For this example, we can use the data above next, to use the method allowing to look into a specific campaign if you have the information (name) already.

In [ ]:
project = survey_campaigns["data"]["projectsite_name"].iloc[0]
survey_campaign = survey_campaigns["data"]["title"].iloc[0]

In [ ]:
survey_campaigns_details = api_geo.get_surveycampaign_detail(projectsite=project, campaign=survey_campaign)
survey_campaigns_details["data"]

To show the closest locations based on longitude and latitude for a given radius, we run the following:

In [ ]:
close_loc = api_geo.get_proximity_testlocations(latitude=51.5, longitude=2.8, radius=10.0)
if close_loc["exists"]:
    display(close_loc["data"])
else:
    print("No locations found within the radius or/and at this location.")

Please note the outcome of this search depends on to what you have access to. In case of failure to find anything it usually returns an empty dataframe and False exists value. Additionaly, you can use specific method to get only the closest location using the above input data as well.

Some types of data allow for explicit checking if the specific element exists, e.g. for test locations:

In [ ]:
close_loc = api_geo.testlocation_exists(projectsite="Nobelwind", campaign="Borehole campaign", location="CPT-888")
close_loc["data"]

More complex data will provide more extensive output to the requests, e.g. to get cpt test details:

In [ ]:
res = api_geo.get_cpttest_detail(insitutest="CPT-888")
display(res["id"])
display(res["insitutestsummary"])
display(res["rawdata"])
display(res["processeddata"])

There are much more different methods which can be queried according to the documentation.

### Final remarks

The package is currently a work in progress. In case of issues (code, docs)/bugs/suggestions contact the [authors](mailto:Arsen.Melnikov@vub.be) or [file an issue on GitHub](https://github.com/OWI-Lab/owimetadatabase-preprocessor/issues).

For more specific information about the explained functionality or more functionality, please [visit the documentation](https://owi-lab.github.io/owimetadatabase-preprocessor/index.html).